In [10]:
import itertools
import numpy as np

In [11]:
# function scoring_matrix(), is used to set and calculate the scoring matrix.
def scoring_matrix(seq1, seq2, match, mismatch, gap_penalty):
    # generate a matrix with zeros, size is (seq1+1)*(seq2+1).
    matrix = np.zeros((len(seq1) + 1, len(seq2) + 1), int)

    max_score = 0
    max_pos = 0
    for i, j in itertools.product(range(1, matrix.shape[0]), range(1, matrix.shape[1])):
        # if bases in the position[i-1] and [j-1] match each other, add match score.
        if seq1[i - 1] == seq2[j - 1]:
            match_score = matrix[i - 1, j - 1] + match
        # if bases in the position[i-1] and [j-1] do not match each other, substract mismatch score.
        else:
            match_score = matrix[i - 1, j - 1] - mismatch
        # return the value in the position[i-1,j] and [i,j-1], substract gap_penalty.
        deletion = matrix[i - 1, j] - gap_penalty
        insertion = matrix[i, j - 1] - gap_penalty
        # take the largest value among match_score, deletion, insertion and zero.
        score = max(match_score, deletion, insertion, 0)
        matrix[i, j] = score
        # find the position that has largest value in the matrix to traceback.
        if score > max_score:
            max_score = score
            max_pos = (i, j)
    print (matrix)
    return matrix, max_pos

In [12]:
# function next_move() is used to decide the next move when traceback(return as a value).
def next_move(matrix, x, y):
    diagonal = matrix[x-1][y-1]
    up = matrix[x-1][y]
    left = matrix[x][y-1]
    if diagonal >= up and diagonal >= left:
        if diagonal != 0:
            # 1 means go to diagonal direction.
            return 1
            # 0 means end. no more movement.
        else: return 0
    elif up > diagonal and up >= left:
        if up != 0:
            # 2 means go to up direction.
            return 2
        else: return 0
    elif left > diagonal and left > up:
        if left != 0:
            # 3 means go to left direction.
            return 3
        else: return 0
    else:
        print("error")

In [13]:
# function traceback() is used to traceback from the calculated scoring matrix, to get the optimal alignment of seq1 with seq2.
def traceback(seq1, seq2, matrix, start_pos):
    # lists aligned_seq1, aligned_seq2 are used to save the aligned results.
    aligned_seq1 = []
    aligned_seq2 = []
    x, y = start_pos
    move = next_move(matrix, x, y)
    # if next move is end, break while loop.
    while move != 0:
        # if next move is diagonal direction.
        if move == 1:
            aligned_seq1.append(seq1[x-1])
            aligned_seq2.append(seq2[y-1])
            x = x - 1
            y = y - 1
        # if next move is up direction.
        elif move == 2:
            aligned_seq1.append(seq1[x-1])
            aligned_seq2.append('-')
            x = x - 1
        # if next move is left direction.
        else:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[y-1])
            y = y - 1

        move = next_move(matrix, x, y)
    
    aligned_seq1.append(seq1[x - 1])
    aligned_seq2.append(seq2[y - 1])

    return ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))

In [14]:
def smith_waterman(seq1, seq2, match, mismatch, gap_penalty):
    # calculate and generate scoring matrix.
    matrix, start_pos = scoring_matrix(seq1, seq2, match, mismatch, gap_penalty)
    # traceback to get the optimal alignment of seq1 with seq2.
    seq1_aligned, seq2_aligned = traceback(seq1, seq2, matrix, start_pos)
    # final score is the value in the position start_pos of the matrix.
    final_score = matrix[start_pos[0]][start_pos[1]]
    print ("aligned seq1 = ", seq1_aligned)
    print ("aligned seq2 = ", seq2_aligned)
    print ("score is" , final_score)

In [15]:
# testing code1.
# seq1 = 'tgcatcgagaccctacgtgac', 
# seq2 = 'actagacctagcatcgac'
# score = 8 (true for all results)
smith_waterman('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', 1, 1, 1)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 2 1 0 1 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 1 0 1 3 2 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 2 4 3 2 1 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 1 1 3 5 4 3 2]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 2 4 6 5 4]
 [0 1 0 0 1 0 2 1 0 0 1 0 0 1 1 3 5 7 6]
 [0 0 0 0 0 2 1 1 0 0 0 2 1 0 0 2 4 6 6]
 [0 1 0 0 1 1 3 2 1 0 1 1 1 2 1 1 3 5 5]
 [0 0 2 1 0 0 2 4 3 2 1 0 2 1 1 2 2 4 6]
 [0 0 1 1 0 0 1 3 5 4 3 2 1 1 0 2 1 3 5]
 [0 0 1 0 0 0 0 2 4 4 3 2 3 2 1 1 1 2 4]
 [0 0 0 2 1 0 0 1 3 5 4 3 2 2 3 2 1 1 3]
 [0 1 0 1 3 2 1 0 2 4 6 5 4 3 2 2 1 2 2]
 [0 0 2 1 2 2 1 2 1 3 5 5 6 5 4 3 2 1 3]
 [0 0 1 1 1 3 2 1 1 2 4 6 5 5 4 3 4 3 2]
 [0 0 0 2 1 2 2 1 0 2 3 5 5 4 6 5 4 3 2]
 [0 0 0 1 1 2 1 1 0 1 2 4 4 4 5 5 6 5 4]
 [0 1 0 0 2 1 3 2 1 0 2 3 3 5 4 4 5 7 6]
 [0 0 2 1 1 1 2 4 3 2 1 2 4 4 4 5 4 6 8]]
aligned seq1 =  agacccta-cgt-gac
aligned seq2 =  agacc-tagcatcgac
score is 8


In [16]:
# testing code2.
smith_waterman('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', 1, 1, 2)
# there is only one possibility: seq1 = seq2 = gcatcga, score=7

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 2 0 0 1 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 1 0 0 3 1 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 1 4 2 0 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 1 0 2 5 3 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 3 6 4 2]
 [0 1 0 0 1 0 2 0 0 0 1 0 0 1 0 1 4 7 5]
 [0 0 0 0 0 2 0 1 0 0 0 2 0 0 0 0 2 5 6]
 [0 1 0 0 1 0 3 1 0 0 1 0 1 1 0 0 0 3 4]
 [0 0 2 0 0 0 1 4 2 0 0 0 1 0 0 1 0 1 4]
 [0 0 1 1 0 0 0 2 5 3 1 0 1 0 0 1 0 0 2]
 [0 0 1 0 0 0 0 1 3 4 2 0 1 0 0 1 0 0 1]
 [0 0 0 2 0 0 0 0 1 4 3 1 0 0 1 0 0 0 0]
 [0 1 0 0 3 1 1 0 0 2 5 3 1 1 0 0 0 1 0]
 [0 0 2 0 1 2 0 2 1 0 3 4 4 2 0 1 0 0 2]
 [0 0 0 1 0 2 1 0 1 0 1 4 3 3 1 0 2 0 0]
 [0 0 0 1 0 0 1 0 0 2 0 2 3 2 4 2 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1 1 1 2 2 3 3 1 0]
 [0 1 0 0 1 0 2 0 0 0 1 0 0 2 1 1 2 4 2]
 [0 0 2 0 0 0 0 3 1 0 0 0 1 0 1 2 0 2 5]]
aligned seq1 =  gcatcga
aligned seq2 =  gcatcga
score is 7


In [17]:
# testing code3.
smith_waterman('tgccgttgaatcg', 'ctccttggaacg', 1, 1, 1)

[[0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 1 0 0 0 1]
 [0 1 0 1 1 0 0 1 1 0 0 1 0]
 [0 1 0 1 2 1 0 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 1 1 0 0 0 2]
 [0 0 1 0 0 2 2 1 0 0 0 0 1]
 [0 0 1 0 0 1 3 2 1 0 0 0 0]
 [0 0 0 0 0 0 2 4 3 2 1 0 1]
 [0 0 0 0 0 0 1 3 3 4 3 2 1]
 [0 0 0 0 0 0 0 2 2 4 5 4 3]
 [0 0 1 0 0 1 1 1 1 3 4 4 3]
 [0 1 0 2 1 0 0 0 0 2 3 5 4]
 [0 0 0 1 1 0 0 1 1 1 2 4 6]]
aligned seq1 =  ccgttg-aatcg
aligned seq2 =  cc-ttggaa-cg
score is 6


In [19]:
# testing code3.
smith_waterman('tgccgttgaatcg', 'ctccttggaacg', 2, 2, 2)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  2  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  4  2  0  0  0  2]
 [ 0  2  0  2  2  0  0  2  2  0  0  2  0]
 [ 0  2  0  2  4  2  0  0  0  0  0  2  0]
 [ 0  0  0  0  2  2  0  2  2  0  0  0  4]
 [ 0  0  2  0  0  4  4  2  0  0  0  0  2]
 [ 0  0  2  0  0  2  6  4  2  0  0  0  0]
 [ 0  0  0  0  0  0  4  8  6  4  2  0  2]
 [ 0  0  0  0  0  0  2  6  6  8  6  4  2]
 [ 0  0  0  0  0  0  0  4  4  8 10  8  6]
 [ 0  0  2  0  0  2  2  2  2  6  8  8  6]
 [ 0  2  0  4  2  0  0  0  0  4  6 10  8]
 [ 0  0  0  2  2  0  0  2  2  2  4  8 12]]
aligned seq1 =  ccgttg-aatcg
aligned seq2 =  cc-ttggaa-cg
score is 12
